In [9]:
import os
import json
import math

In [52]:
trainPath = "./Processed_text"
testPath = "./test"
fileExtension = ".txt"
outPathText = "./LM_addK"
outPathJson = "./Output"
word_count = {}
unigram_count = {}
bigram_count = {}
trigram_count = {}
V = 0
K = 0.1

In [53]:
def createUnigrams(wordList,outDict):
    for i in range(0, len(wordList)):
        unigram = wordList[i]
        if unigram in outDict:
            outDict[unigram] = outDict[unigram] + 1
        else:
            outDict[unigram] = 1

In [54]:
def createBigrams(wordList,outDict):
    for i in range(0, len(wordList) - 1, 2):
        bigram = wordList[i] + " " + wordList[i + 1]
        if bigram in outDict:
            outDict[bigram] = outDict[bigram] + 1
        else:
            outDict[bigram] = 1

In [55]:
def createTrigrams(wordList,outDict):
    for i in range(0, len(wordList) - 2, 3):
        trigram = wordList[i] + " " + wordList[i + 1] + " " + wordList[i+2]
        if trigram in outDict:
            outDict[trigram] = outDict[trigram] + 1
        else:
            outDict[trigram] = 1

In [56]:
def get_word_count(wordList,outDict):
    for word in wordList:
        if word in outDict:
            outDict[word] = outDict[word] + 1
        else:
            outDict[word] = 1

In [57]:
def get_word_list(path, fileName):
    fp = open(path + "/" + fileName, "r+")
    print("Getting word count...." + fileName)
    text = fp.read()
    text = text.replace("\n\n", " </s> <s> ")
    text = text.replace("."," . ")
    text = text.replace(","," , ")
    text = text.replace("?"," ? ")
    text = text.strip().strip()
    text = " ".join(text.split()).lower()
    wordList = text.split()
    return wordList

In [2]:
def process_file_test(wordList):
    processed_text = ""
    for word in wordList:
        if word not in unigram_count:
            processed_text = processed_text + "UNK "
        elif unigram_count[word] <= 3:
            processed_text = processed_text + "UNK "
        else:
            processed_text = processed_text + word + " "
    return processed_text

In [1]:
def calculate_perplexity(test_text,K):
    l1 = lambdas[0]
    l2 = lambdas[1]
    l3 = lambdas[2]
    test_text = test_text.strip()
    wordList = test_text.split(" ")
    wordList = filter(lambda x: len(x) != 0, wordList) 
    totalWords = 0
    log_trigram_prob = 0
    for i in range(0,len(wordList)-3):
        n0 = wordList[i]
        n1 = wordList[i+1]
        n2 = wordList[i+2]
        if n1 + " " + n2 in bigram_count:
            bigram_C = float(bigram_count[n1+ " " + n2])
        else :
            bigram_C = 0
        if n0 + " " + n1 + " " + n2 in trigram_count:
            trigram_C = float(trigram_count[n0 + " " + n1 + " " + n2])
        else:
            trigram_C = 0
        log_trigram_prob = log_trigram_prob + math.log((trigram_C+K),2)-math.log(((bigram_C)+K*V),2)
        totalWords += 1
        log_perplexity = -1 * log_trigram_prob / totalWords
    perplexity = math.pow(2,log_perplexity)
    print("K: " + str(K) + " " + "Perplexity :" + str(perplexity))
    return perplexity

In [62]:
fileList = os.listdir(trainPath)
for file in fileList:
    if file.endswith(fileExtension):
        print("Processing......" + file)
        fp = open(trainPath + "/" + file, "r+")
        txt = fp.read()
        wordList_UNK = txt.split()
        createUnigrams(wordList_UNK,unigram_count)
        createBigrams(wordList_UNK,bigram_count)
        createTrigrams(wordList_UNK,trigram_count)

Processing......9.txt
Processing......8.txt
Processing......5.txt
Processing......4.txt
Processing......6.txt
Processing......7.txt
Processing......3.txt
Processing......2.txt
Processing......0.txt
Processing......1.txt


In [63]:
V = len(unigram_count)

In [64]:
print("Testing.......")
print("========================")
for f in os.listdir(testPath):
    test_text = ""
    if f.endswith(fileExtension):
        wordList = get_word_list(testPath,f)
        test_text = process_file_test(wordList)
    pp = calculate_perplexity(test_text,K)
    print("File : " + f + " Perplexity: " + str(pp))

Testing.......
Getting word count....test01.txt
K: 0.1 Perplexity :2810.98201059
File : test01.txt Perplexity: 2810.98201059
Getting word count....test02.txt
K: 0.1 Perplexity :1560.28672344
File : test02.txt Perplexity: 1560.28672344
